# Custom satellites

In [1]:
import os
from skyfield.api import load
import numpy as np
from skyfield.api import load, EarthSatellite

## Modify parameters of an existing satellite

In [2]:
resource_url = 'http://celestrak.com/NORAD/elements/resource.txt'

fname = 'tle.txt'
if os.path.isfile(fname):
    os.remove(fname)

satellites = load.tle(resource_url, filename=fname, reload=True)
satellite = satellites['HODOYOSHI-1']
print(satellite)

[#################################] 100% tle.txt


EarthSatellite 'HODOYOSHI-1' number=40299 epoch=2018-08-06T18:53:38Z


In [3]:
print(satellite.epoch.utc_jpl())

A.D. 2018-Aug-06 18:53:37.7209 UT


### Modify parameters

In [4]:
print('Inclination: {:}'.format(180 - np.rad2deg(satellite.model.inclo)))

Inclination: 82.6238


In [5]:
# change inclination to 15 degrees
deg = 15
satellite.model.inclo = np.deg2rad(180 - deg)

In [6]:
print('Inclination: {:}'.format(180 - np.rad2deg(satellite.model.inclo)))

Inclination: 15.0


## Create satellite from custom TLE elements

In [7]:
text = """
GOCE
1 34602U 09013A   13314.96046236  .14220718  20669-5  50412-4 0   930
2 34602 096.5717 344.5256 0009826 296.2811 064.0942 16.58673376272979
"""
lines = text.strip().splitlines()

custom_satellite = EarthSatellite(lines[1], lines[2], lines[0])

## Sources

- http://rhodesmill.org/skyfield/api-satellites.html#skyfield.sgp4lib.EarthSatellite
- https://en.wikipedia.org/wiki/Two-line_element_set